# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [2]:
import pandas as pd
import logging
import json
from utilities import text_to_json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [3]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()
        responses = scoring_func(inputString)
        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Overwriting driver.py


Let's test the module.

In [4]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [5]:
%run driver.py

/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Now, let's use one of the duplicate questions to test our driver.

In [6]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

"getting the length of a 'named' array?. i'm not sure what they are called, but what i mean is this:  length should be 2 here how can i see how many attributes i have in the array? array.length doesn't work =( i've been trying all kinds of things and i feel like i'm missing something really simple here.. thank you for your help"

Here, we convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [7]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [8]:
predict_for = get_model_api()

INFO:model_driver:Model object loading time: 401.3 ms


In [9]:
resp = predict_for(body)

INFO:model_driver:Predictions: [(5223, 6700, 0.9056762423642606), (126100, 4889658, 0.5897041951019686), (7364150, 7364307, 0.013095757762535671), (19590865, 19590901, 0.009726747791608314), (11922383, 11922384, 0.007400362725375734), (5627284, 5627301, 0.006518820196008976), (1771786, 1771824, 0.004990243605901752), (24900875, 24900924, 0.004120260321857448), (7837456, 14853974, 0.0016726596738734054), (1129216, 1129270, 0.0014618991064883388), (237104, 1473742, 0.0009983357055414952), (784929, 784946, 0.000918666929043429), (23667086, 23667087, 0.0007934756119722436), (4616202, 4616273, 0.0006798101809810278), (750486, 750506, 0.0005087117457724936), (14028959, 8716680, 0.0004299633021580329), (14220321, 14220323, 0.00037730639500855277), (85992, 86014, 0.00030718180657696116), (695050, 695053, 0.00029608261688253765), (1822350, 1822769, 0.00024789271341276657), (10693845, 10693852, 0.00022361713661220404), (5767325, 5767357, 0.00019449493626948474), (1068834, 1144249, 0.000167068897

Next, we move on to [building our docker image](03_Build_Image.ipynb).